# SkySat-snow pipeline: coregister and difference RASTERS

In [2]:
import os
import sys

# Define paths in directory
code_dir = '/Users/raineyaberle/Research/PhD/SnowDEMs/skysat-snow'
ref_dem_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/MCS/refdem/MCS_REFDEM_WGS84.tif'
pc_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/MCS/20241003/point_clouds'
roads_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/MCS/roads/MCS_roads_line_merged_UTM11N.shp'
multispec_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/MCS/20241003/point_clouds'
asp_dir = '/Users/raineyaberle/Research/PhD/SnowDEMs/StereoPipeline-3.5.0-alpha-2024-10-05-x86_64-OSX/bin'
out_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/MCS/20241003'
res = 2 # spatial resolution of outputs [m]
roads_buffer = 3 # buffer for roads geospatial file [m]

# Check that input files and directories exist
if not os.path.exists(ref_dem_fn):
    print('Reference DEM file not found, please correct ref_dem_fn before continuing.')
if not os.path.exists(pc_dir):
    print('Path to point clouds not found, please correct pc_dir before continuing.')
if not os.path.exists(roads_fn):
    print('Roads geospatial file not found, please correct roads_fn before continuing.')
if not os.path.exists(multispec_dir):
    print('Multispectral iamges folder not found, please correct multispec_dir before continuing.')
if not os.path.exists(asp_dir):
    print('Path to ASP not found, please correct asp_dir before continuing.')
out_dir = os.path.join(out_dir, 'ice-road-skysat')
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    print('Made directory for outputs:', out_dir)

# Make results directories
preprocess_dir = os.path.join(out_dir, 'preprocess')
masks_dir = os.path.join(out_dir, 'land_cover_masks')
deramp_dir = os.path.join(out_dir, 'deramp_init')
coreg_init_dir = os.path.join(out_dir, 'coreg_diff_init')
postprocess_dir = os.path.join(out_dir, 'postprocess')
for folder in [preprocess_dir, masks_dir, deramp_dir, coreg_init_dir, postprocess_dir]:
    if not os.path.exists(folder):
        os.mkdir(folder)

# Add path to pipeline utilities
sys.path.append(os.path.join(code_dir, 'skysat-snow'))
import pipeline_utils_raster as f

## 1. Preprocessing

In [ ]:
# Merge and filter point clouds
pc_filtered_las_fn, pc_filtered_tif_fn = f.preprocess_point_clouds(pc_dir, ref_dem_fn, res, preprocess_dir)

# Mosaic 4-band SR imagery, preprocess roads and reference DEM
multispec_mosaic_fn, roads_adj_fn, ref_dem_adj_fn = f.preprocess_multispec_refdem_roads(multispec_dir, ref_dem_fn, roads_fn, roads_buffer, res, out_dir)


## 2. Construct land cover masks

In [ ]:
multispec_mosaic_fn, trees_mask_fn, snow_mask_fn, roads_mask_fn, ss_mask_fn = f.construct_land_cover_masks(multispec_mosaic_fn, 
                                                                                                           roads_adj_fn, 
                                                                                                           masks_dir, 
                                                                                                           roads_buffer,
                                                                                                           ndvi_threshold=0.2)

## 3. Initial deramping

In [ ]:
dem_deramped_init_fn = f.deramp(ref_dem_fn=ref_dem_fn, 
                                tba_dem_fn=pc_filtered_tif_fn, 
                                ss_mask_fn=ss_mask_fn, 
                                out_dir=deramp_init_dir,
                                vmin=-10, vmax=10)

## 4. Initial coregistration

In [ ]:
dem_coreg_init_fn = f.coregister_difference_dems(ref_dem_fn=ref_dem_adj_fn, 
                                                 source_dem_fn=dem_deramped_init_fn, 
                                                 ss_mask_fn=ss_mask_fn, 
                                                 out_dir=coreg_diff_init_dir, 
                                                 coreg_method='NuthKaab', 
                                                 coreg_stable_only=True, 
                                                 vmin=-5, vmax=5)

## 5. Final deramping

In [ ]:
dem_deramped_final_fn = f.deramp(ref_dem_fn=ref_dem_adj_fn, 
                                 tba_dem_fn=dem_coreg_init_fn, 
                                 ss_mask_fn=ss_mask_fn,
                                 out_dir=deramp_final_dir,
                                 vmin=-5, vmax=5)

## 6. Final coregistration and differencing

In [ ]:
dem_coreg_final_fn = f.coregister_difference_dems(ref_dem_fn=ref_dem_adj_fn, 
                                                  source_dem_fn=dem_deramped_final_fn, 
                                                  ss_mask_fn=ss_mask_fn, 
                                                  out_dir=coreg_diff_final_dir, 
                                                  coreg_method='NuthKaab', 
                                                  coreg_stable_only=True, 
                                                  vmin=-5, vmax=5)